In [ ]:
# Install required packages
!pip install tweepy schedule transformers torch requests

In [ ]:
import os
import json
import logging
import random
import torch
import warnings
import time
from typing import List, Dict, Optional, Set
from transformers import pipeline, logging as transformers_logging
import tweepy

# Twitter API Credentials
API_KEY = "lgKfDigC7qbos2zUIQeCvfbph"
API_SECRET_KEY = "NkkFO9wCVvsvv4m6GOssoXC0rCcE4qUav0rvVvM5xhWuJIkod2"
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAALbPzwEAAAAA55b7v5kLBGPx1JyibiHmBQpmxoo%3Dhzw1o30lEajXtkuinTH1IAmjnIE4pILWy8e1wklNvWRA6N8j7f"
ACCESS_TOKEN = "1674821745340211200-UQzdIVGRCTSt7D5411vpxV9ApP9msO"
ACCESS_TOKEN_SECRET = "EjwT4klBT9nelP3aDonZz6hcruZ6IciO987TpRnnPFkif"
CLIENT_ID = "UFAwM1M1aWs2TVgyZ2F6R3Z4RzU6MTpjaQ"
CLIENT_SECRET = "yy0c-4hcra6OJz0RodTF60gWAg2Uj802bWKUl3XErsUzJUb06n"

# Suppress warnings and non-critical logging
warnings.filterwarnings("ignore")
transformers_logging.set_verbosity_error()
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

class TweetGenerator:
    def __init__(
        self,
        model_name: str = "distilgpt2",  # Lightweight, fast model
        topics_file: str = "topics.json",  # Source of tweet prompts
        max_tweet_length: int = 280,  # X's max tweet length
        cache_dir: Optional[str] = None  # Unused cache option
    ):
        """Initialize the TweetGenerator with model and topics."""
        self.topics_file = topics_file
        self.max_tweet_length = max_tweet_length
        self.model_name = model_name
        self.cache_dir = cache_dir
        self.topics = self._load_static_topics()  # Load or create topics
        self.used_topics: Set[int] = set()  # Track used topics
        self._setup_generator()  # Setup AI model
        device = "cuda" if torch.cuda.is_available() else "cpu"
        logger.info(f"Using device: {device}")

    def _setup_generator(self):
        """Initialize the distilgpt2 model pipeline."""
        try:
            logger.info(f"Loading {self.model_name} model...")
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                self.generator = pipeline(
                    "text-generation",
                    model=self.model_name,
                    device_map="auto",
                    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
                )
            logger.info("Model initialized successfully.")
        except Exception as e:
            logger.error(f"Failed to initialize model: {str(e)}")
            raise

    def _load_static_topics(self) -> List[Dict[str, str]]:
        """Load or create default topics from/to topics.json."""
        try:
            if not os.path.exists(self.topics_file):
                sample_topics = [
                    {"topic": "Future of AI", "prompt": "AI is evolving beyond automation, unlocking"},
                    {"topic": "Immersive Gaming", "prompt": "Gaming is shifting towards hyper-realistic worlds with"},
                    {"topic": "Decentralized Finance", "prompt": "DeFi is transforming traditional banking by"},
                    {"topic": "Code Optimization", "prompt": "Writing efficient code isn't just about speed, it's about"},
                    {"topic": "Next-Gen Cybersecurity", "prompt": "Tomorrow's digital threats demand proactive security like"},
                    {"topic": "The Rise of Hybrid Work", "prompt": "Blending remote and in-office work successfully depends on"},
                    {"topic": "AI in Marketing", "prompt": "AI-driven marketing isn't just about ads, it's about"},
                    {"topic": "The App Revolution", "prompt": "Smart apps are becoming personal assistants by"}
                ]
                dir_path = os.path.dirname(os.path.abspath(self.topics_file))
                if dir_path and not os.path.exists(dir_path):
                    os.makedirs(dir_path, exist_ok=True)
                with open(self.topics_file, 'w') as f:
                    json.dump(sample_topics, f, indent=2)
                logger.info(f"Created new topics file with {len(sample_topics)} topics")
            with open(self.topics_file, 'r') as f:
                topics = json.load(f)
            logger.info(f"Loaded {len(topics)} topics from {self.topics_file}")
            return topics
        except Exception as e:
            logger.error(f"Failed to load static topics: {str(e)}")
            raise

    def _get_next_topic(self) -> Optional[Dict[str, str]]:
        """Retrieve a random unused topic, reset if all used."""
        if not self.topics:
            logger.error("No topics available")
            return None
        if len(self.used_topics) >= len(self.topics):
            logger.info("All topics have been used. Resetting.")
            self.used_topics.clear()
        available_topics = [i for i in range(len(self.topics)) if i not in self.used_topics]
        if available_topics:
            topic_index = random.choice(available_topics)
            self.used_topics.add(topic_index)
            return self.topics[topic_index]
        return None

    def _clean_tweet(self, text: str) -> str:
        """Clean text to a single, concise sentence under 280 chars."""
        text = text.strip()  # Remove extra whitespace
        sentences = text.split('.')  # Split into sentences
        if len(sentences) > 1:  # Take only first sentence
            text = sentences[0].strip() + '.'
        text = ''.join(c for c in text if c.isalnum() or c in ' .,!?')  # Keep valid chars
        if text and text[0].islower():  # Capitalize first letter
            text = text[0].upper() + text[1:]
        if text and not text[-1] in ['.', '!', '?']:  # Add ending punctuation
            text += '.'
        if len(text) > self.max_tweet_length:  # Truncate if too long
            text = text[:self.max_tweet_length-1] + '.'
        return text

    def _generate_tweet_content(self, topic: Dict[str, str]) -> Optional[str]:
        """Generate a clean, concise tweet from a topic prompt."""
        try:
            prompt = topic['prompt']  # Use prompt as base
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                outputs = self.generator(
                    prompt,
                    max_new_tokens=30,  # Reduced for conciseness (was 50)
                    temperature=0.7,  # Lowered for coherence (was 0.85)
                    top_p=0.9,  # Tighter for focus (was 0.92)
                    num_return_sequences=1,
                    do_sample=True,
                    pad_token_id=50256
                )
            generated_text = outputs[0]['generated_text']  # Get raw output
            tweet = self._clean_tweet(generated_text)  # Clean to desired format
            logger.info(f"Generated tweet content: {tweet} ({len(tweet)} characters)")
            return tweet
        except Exception as e:
            logger.error(f"Failed to generate tweet content: {str(e)}")
            return None

    def generate_tweet(self) -> Optional[str]:
        """Generate a single tweet from a random topic."""
        topic = self._get_next_topic()
        if not topic:
            logger.warning("No topic available")
            return None
        tweet_content = self._generate_tweet_content(topic)
        if not tweet_content:
            logger.warning("Failed to generate tweet content")
            return None
        logger.info(f"Generated tweet ({len(tweet_content)} chars): {tweet_content}")
        return tweet_content

    def generate_engaging_tweet(self) -> Optional[str]:
        """Generate an engaging tweet, picking the best from 3 attempts."""
        best_tweet = None
        best_length = 0
        for _ in range(3):  # Try 3 times for quality
            topic = self._get_next_topic()
            if not topic:
                continue
            tweet = self._generate_tweet_content(topic)
            if not tweet:
                continue
            tweet_length = len(tweet)
            if (best_tweet is None) or (60 <= tweet_length <= 200 and tweet_length > best_length):  # Adjusted range for conciseness
                best_tweet = tweet
                best_length = tweet_length
        if best_tweet:
            logger.info(f"Selected best tweet: {best_tweet}")
            return best_tweet
        else:
            return self.generate_tweet()  # Fallback if no good tweet

    def generate_tweets_batch(self, count: int = 3) -> List[str]:
        """Generate multiple tweets (unused here)."""
        tweets = []
        for _ in range(count):
            tweet = self.generate_tweet()
            if tweet:
                tweets.append(tweet)
        return tweets

    def add_topic(self, topic: str, prompt: str) -> bool:
        """Add a new topic to the file for future use."""
        try:
            new_topic = {"topic": topic, "prompt": prompt}
            self.topics.append(new_topic)
            with open(self.topics_file, 'w') as f:
                json.dump(self.topics, f, indent=2)
            logger.info(f"Added new topic: {topic}")
            return True
        except Exception as e:
            logger.error(f"Failed to add topic: {str(e)}")
            return False

def post_tweet(client, tweet):
    """Post a tweet using the X v2 API."""
    try:
        response = client.create_tweet(text=tweet)
        logger.info(f"Tweet posted successfully with ID: {response.data['id']}")
        print(f"\nSuccessfully posted tweet: {tweet} (Tweet ID: {response.data['id']})")
    except tweepy.TweepyException as e:
        logger.error(f"Failed to post tweet: {str(e)}")
        print(f"\nFailed to post tweet: {str(e)}")

def main():
    """Generate and post two tweets with a 2-minute interval."""
    print("\n Initializing Tweet Generator...")
    generator = TweetGenerator()

    client = tweepy.Client(
        bearer_token=BEARER_TOKEN,
        consumer_key=API_KEY,
        consumer_secret=API_SECRET_KEY,
        access_token=ACCESS_TOKEN,
        access_token_secret=ACCESS_TOKEN_SECRET
    )

    # First tweet
    print("\n Generating first engaging tweet...")
    tweet1 = generator.generate_engaging_tweet()
    if tweet1:
        print("\n FIRST ENGAGING TWEET ")
        print(f"{tweet1}")
        post_tweet(client, tweet1)
    else:
        print("Failed to generate first engaging tweet.")

    # Wait 2 minutes
    print("\nWaiting before posting the second tweet...")
    time.sleep(120)

    # Second tweet
    print("\n Generating second engaging tweet...")
    tweet2 = generator.generate_engaging_tweet()
    if tweet2:
        print("\n SECOND ENGAGING TWEET ")
        print(f"{tweet2}")
        post_tweet(client, tweet2)
    else:
        print("Failed to generate second engaging tweet.")

    logger.info("Two-tweet generation and posting process completed.")
    print("\nProcess completed.")

if __name__ == "__main__":
    main()